In [ ]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from heapq import nlargest
import gensim
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from string import punctuation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
url="https://habr.com/ru/news/727002/"
response=requests.get(url)
response

<Response [200]>

In [ ]:
soup=BeautifulSoup(response.content,"html.parser")
title=soup.find('h1').text
title

'Робот Boxing Buddy позволяет боксёрским грушам давать отпор'

In [ ]:
article_text=''
for p in soup.find_all('p'):
  article_text+=p.text

In [ ]:
article_text[:50]

'Компания Boxing Buddy представила одноимённый трен'

In [ ]:
sentences=sent_tokenize(article_text)
words=word_tokenize(article_text)
num_sentences=len(sentences)

In [ ]:
sentences[:5]

['Компания Boxing Buddy представила одноимённый тренажёр, который даёт боксёрским грушам возможность бить в ответ.',
 'Тренажёр доступен на краудфандинговой площадке Indiegogo за $250.«Boxing Buddy — идеальный интеллектуальный помощник для всех любителей бокса.',
 'Это портативное дополнение превращает вашу боксёрскую грушу в спарринг-партнёра, который наносит ответный удар, делая ваши тренировки более сложными и полными», — пишут разработчики.Установка Boxing Buddy состоит из двух мягких роборук, которые присоединяются к боксёрской груше на любой желаемой высоте.',
 'Руки также можно закрепить на дереве, тренировочном манекене или любом другом прочном вертикальном объекте.',
 'Тренажёр подключается к смартфону через Bluetooth.']

In [ ]:
stop_words=set(stopwords.words("russian")+list(punctuation))
filtered_words=[]

for word in words:
  if word.casefold() not in stop_words:
    filtered_words.append(word)

In [ ]:
word_freqs={}

for word in filtered_words:
  if word not in word_freqs:
    word_freqs[word]=1
  else:
    word_freqs[word]+=1

In [ ]:
max_freq=max(word_freqs.values())
for word in word_freqs.keys():
  word_freqs[word]/=max_freq

In [ ]:
sent_scores={}

for sentence in sentences:
  for word in word_tokenize(sentence.lower()):
    if word in word_freqs.keys():
      if len(sentence.split(" "))<30:
        if sentence not in sent_scores.keys():
          sent_scores[sentence]=word_freqs[word]
        else:
          sent_scores[sentence]+=word_freqs[word]

In [ ]:
sent_scores

{'Компания Boxing Buddy представила одноимённый тренажёр, который даёт боксёрским грушам возможность бить в ответ.': 2.4000000000000004,
 'Тренажёр доступен на краудфандинговой площадке Indiegogo за $250.«Boxing Buddy — идеальный интеллектуальный помощник для всех любителей бокса.': 3.2000000000000006,
 'Руки также можно закрепить на дереве, тренировочном манекене или любом другом прочном вертикальном объекте.': 2.1999999999999997,
 'Тренажёр подключается к смартфону через Bluetooth.': 0.6000000000000001,
 'В приложении также есть функция «Школа» с серией обучающих видеороликов.Вся система весит около 2 кг.': 3.2000000000000006,
 'Одной 1,5-часовой зарядки тренажёра хватит на 10 часов работы.': 1.4,
 'Планируется, что Boxing Buddy поступит в продаже по цене в $449.': 0.8,
 'Сейчас он доступен на краудфандинговой платформе за $249.Информационная служба Хабра': 1.2}

In [ ]:
summary_sents=nlargest(5,sent_scores,key=sent_scores.get)
summary=" ".join(summary_sents)

In [ ]:
print(summary)

Тренажёр доступен на краудфандинговой площадке Indiegogo за $250.«Boxing Buddy — идеальный интеллектуальный помощник для всех любителей бокса. В приложении также есть функция «Школа» с серией обучающих видеороликов.Вся система весит около 2 кг. Компания Boxing Buddy представила одноимённый тренажёр, который даёт боксёрским грушам возможность бить в ответ. Руки также можно закрепить на дереве, тренировочном манекене или любом другом прочном вертикальном объекте. Одной 1,5-часовой зарядки тренажёра хватит на 10 часов работы.


In [ ]:
my_file=open("/content/sample_data/output.txt","w+")
my_file.write(summary)
my_file.close()